In [81]:
%load_ext autoreload
%autoreload 2
import pymongo, math, sys, pandas, itertools
from usta_tennis.utils import *
from usta_tennis.crd import crd
from usta_tennis.drv import drv
import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
client = pymongo.MongoClient('mongodb://localhost:27017')
players, matches, tlsentries = client['usta'].players, client['usta'].matches, client['usta'].tlsentries
tlsentries.count()

801400

In [57]:
entries = tlsentries.aggregate([
        {'$match': {'section': 'No. California', 'area': 'East Bay (EB)', 'year': 2016}},
        {'$group': {'_id': '$name', 'num': {'$sum': 1}}},
        {'$sort': {'num': -1}},
    ])
sd_keys, wl_keys = ['singles', 'doubles'], ['winner', 'loser']
last_names = [entry['_id'].split()[-1] for entry in entries]
player_ids = list(map(int, players.find({'last_name': {'$in': last_names}}).distinct('_id')))

In [78]:
for player_id in player_ids:
    query = {'$or': []}
    for key in itertools.product(sd_keys, wl_keys):
        query['$or'].append({'.'.join(key): player_id})
    player_matches = list(matches.find(query).sort('date'))
    if not player_matches:
        continue
    player = players.find_one({'_id': str(player_id)})
    name = ' '.join([player['first_name'], player['last_name']])
    player_entries = tlsentries.find({'name': name, 'year': 2016})
    nmatches = sum([sum(entry['matches'].values()) for entry in player_entries])
    if nmatches != len(player_matches):
        continue
    print(player_id, name, nmatches)
    break

179173 Kim Iliffe 8


In [79]:
player_id, name = 179173, 'Kim Iliffe'
player = players.find_one({'_id': str(player_id)})
player_entries = tlsentries.find({'name': name, 'year': 2016})
print(name, list(player_entries)[0]['rating'])
print(player_matches)

Kim Iliffe 4.82
[{'singles': [{'score': ['62', '61'], 'winner': 105100, 'loser': 221662}], '_id': 441855, 'doubles': [{'score': ['60', '62'], 'winner': [187233, 179173], 'loser': [61893, 3749]}, {'score': ['75', '76'], 'winner': [11657, 162640], 'loser': [53671, 48839]}], 'date': datetime.datetime(2016, 4, 3, 0, 0)}, {'singles': [{'score': ['62', '63'], 'winner': 204825, 'loser': 117940}], '_id': 442962, 'doubles': [{'score': ['63', '61'], 'winner': [96714, 117756], 'loser': [147546, 88665]}, {'score': ['06', '62', '10'], 'winner': [187233, 179173], 'loser': [77143, 33284]}], 'date': datetime.datetime(2016, 4, 24, 0, 0)}, {'singles': [{'score': ['60', '62'], 'winner': 135506, 'loser': 215452}], '_id': 444058, 'doubles': [{'score': ['62', '63'], 'winner': [69841, 64311], 'loser': [105100, 182033]}, {'score': ['62', '64'], 'winner': [179173, 187233], 'loser': [48874, 202244]}], 'date': datetime.datetime(2016, 5, 1, 0, 0)}, {'singles': [{'score': ['46', '61', '10'], 'winner': 35448, 'lose